In [16]:
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Select user

In [17]:
user_id = 'u_47468' # options are 'u_' + any node number in the sample graph
num_suggestions = 20

Select final models

In [18]:
int_pred_model = 'models/models09062024_2308_LP__loss_bce_tr_True_pcutoff_0.9_poswght_1_bs_16_layers_[16, 32]_nemb_52_lr_0.015_reg_0.005_messdr_0.01_nodedr_0.01_ucount_1000_bcount_500.pt'
scor_class_mdoel = ''

Interaction Prediction

In [19]:
from src.ngcf import NGCF
from src.ngcf_utils.load_data import Data
import networkx as nx
import json
import tqdm
ngcf_link_predictor_folder = "data/ngcf_interaction_predictor/"

In [20]:
# hyperparams from chosen model
loss_name = 'inference'
to_reads = True
prob_cutoff = .90
pos_weight = 1 
batch_size = 16
layers = [16, 32]
emb_dim = 52
reg = 0.005
mess_dropout = 0.01
node_dropout = 0.01

# generate the NGCF-adjacency matrix
data_generator = Data(path=ngcf_link_predictor_folder, batch_size=batch_size)
adj_mtx = data_generator.get_adj_mat()

# pull in idx dictionaries and user + book counts
if to_reads == True:
    train_graph = nx.read_graphml("data/bipartite_sample_train_wtr.graphml")
    test_graph = nx.read_graphml("data/bipartite_sample_test_wtr.graphml")
else:
    train_graph = nx.read_graphml("data/bipartite_sample_train.graphml")
    test_graph = nx.read_graphml("data/bipartite_sample_test.graphml")
with open(ngcf_link_predictor_folder + 'book_to_idx_map.json', "r") as json_file:
    book_to_idx_map = json.load(json_file)
with open(ngcf_link_predictor_folder + 'user_to_idx_map.json', "r") as json_file:
    user_to_idx_map = json.load(json_file)
user_count = len(user_to_idx_map.keys())
book_count = len(book_to_idx_map.keys())
idx_to_book_map = {idx: book_id for book_id, idx in book_to_idx_map.items()}

# Load NGCF model and pretrained embeddings
ngcf_model = NGCF(user_count, book_count, emb_dim, layers, reg, node_dropout, mess_dropout,
                  adj_mtx, pos_weight, prob_cutoff, loss_name)
checkpoint_path = ngcf_link_predictor_folder + int_pred_model
checkpoint = torch.load(checkpoint_path, map_location=torch.device(device))
ngcf_model.load_state_dict(checkpoint, strict=False)
ngcf_model.eval()

# perform inference for a given user
u_idx = user_to_idx_map[user_id]
u_idx = torch.LongTensor([u_idx]).to(device)

interaction_prediction_dict = {}
for i_idx in tqdm.tqdm(range(book_count)):
    i_idx = torch.LongTensor([i_idx]) #.to(device)
    # Forward pass through the model
    with torch.no_grad():
        score = ngcf_model(u_idx, i_idx, i_idx)
    # Convert score to probability (if needed)
    prob = torch.sigmoid(score)
    interaction_prediction_dict[idx_to_book_map[i_idx.item()]] = prob.item()


n_users=1000, n_items=500
n_interactions=33787
n_train=23188, n_test=10599, sparsity=0.06757
Creating interaction matrices R_train and R_test...
Complete. Interaction matrices R_train and R_test created in 0.4000983238220215 sec
Creating adjacency-matrix...
Complete. Adjacency-matrix created in (1500, 1500) 0.0695035457611084 sec.
Transforming adjacency-matrix to NGCF-adjacency matrix...
Complete. Transformed adjacency-matrix to NGCF-adjacency matrix in 0.03026270866394043 sec.


c:\Users\HP\Documents\GitHub\EE452_2024_Team5_Project\src\ngcf_utils\load_data.py:131: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -.5).flatten()


Initializing weights...
Weights initialized.


100%|██████████| 500/500 [00:25<00:00, 19.77it/s]


Rating classification

result should be 'rating_classification_dict' = a dictionary with b_### as keys and star rating as values 

Combination

In [ ]:
final_score_dict = {}
for book_id in interaction_prediction_dict:
    final_score_dict[book_id] = interaction_prediction_dict[book_id] * rating_classification_dict[book_id]

Extraction

In [ ]:
sorted_items = sorted(rating_classification_dict.items(), key=lambda x: x[1], reverse=True)
top_20_keys = [item[0] for item in sorted_items[:20]]
top_20_keys